# Download Meteor M2 satellite info and calculate times for recording

Requires an API key for n2yo.com and username for geonames.org

In [1]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(httr)

In [3]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [4]:
library(jsonlite)


Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    flatten



In [5]:
library(assertthat)


Attaching package: ‘assertthat’

The following object is masked from ‘package:tibble’:

    has_name



In [6]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-15 15:14:50 GMT"

# Helper functions

In [7]:
possibly_get_data <- function(url, n_tries, ...){

    rate <- rate_backoff(pause_base = 1, max_times = n_tries)
    possibly_insistent_get <- insistently(GET, rate, quiet = FALSE) %>% possibly(otherwise = NULL)

    possibly_insistent_get(url, ...)
}

# Config

What satellite are we trying to get orbital data for?

In [8]:
satellite_id <- "METEOR-M2"

In [9]:
satellite_norad_id <- 40069

API key for n2yo.com

In [10]:
n2yo_api_key <- "&apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ"

Username for geonames.org

In [11]:
geonames_username <- "pgcudahy"

Minimal satellite elevation above the horizon before starting data capture (in **degrees**)

In [12]:
minimum_observable_elevation = 20

Station longitude, latitude. Positive values for E, negative for W

In [13]:
station_latitude <- c(-29.53)
station_longitude <- c(30.25)

Get the station's elevation (in meters above sea level)

In [14]:
station_elevation <- possibly_get_data(paste0("http://api.geonames.org/gtopo30JSON?lat=", station_latitude, 
                                   "&lng=", station_longitude, "&username=", geonames_username), 8)

In [15]:
station_elevation <- content(station_elevation)$gtopo30

Sanity check that your coordinates are close to a nearby city

In [16]:
nearby_cities <- possibly_get_data(paste0("http://api.geonames.org/findNearbyPlaceNameJSON?lat=", station_latitude, 
                                   "&lng=", station_longitude, "&username=", geonames_username), 8)

In [17]:
paste0("Station is located near ", content(nearby_cities)$geonames[[1]]$toponymName, ", ",
    content(nearby_cities)$geonames[[1]]$countryName, " at an elevation of ", station_elevation,
    " meters above sea level")

[1] "Station is located near Merrivale, South Africa at an elevation of 1167 meters above sea level"

# Load previous satellite orbital data

In [18]:
satellite_data_df <- tryCatch({
    readRDS("satellite_data_df.rds")}, 
    warning = function(w) {
        print("No prior satellite orbital data found on disk")
        tibble(startAz=double(),
            startAzCompass=character(),
            startUTC=double(),
            maxAz=double(),
            maxAzCompass=character(),
            maxEl=double(),
            maxUTC=integer(),
            endAz=double(),
            endAzCompass=character(),
            endUTC=double(),
            startDate=character(),
            startTime=character(),
            endDate=character(),
            endTime=character(),
            localStartTime=character(),
            duration=double())
}, error = function(e) {
        print("There was an error in trying to load satellite orbital data from disk")
        tibble(startAz=double(),
            startAzCompass=character(),
            startUTC=double(),
            maxAz=double(),
            maxAzCompass=character(),
            maxEl=double(),
            maxUTC=integer(),
            endAz=double(),
            endAzCompass=character(),
            endUTC=double(),
            startDate=character(),
            startTime=character(),
            endDate=character(),
            endTime=character(),
            localStartTime=character(),
            duration=double())
    }
)

# Update satellite orbital data

##  Using the https://www.n2yo.com/api/ webservice since I can't find any R packages to predict satellite passes locally

Request: /radiopasses/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{days}/{min_elevation}
```
Parameter       Type	Required	Comments
id              integer Yes	        NORAD id
observer_lat	float   Yes	        Observer's latitide (decimal degrees format)
observer_lng	float   Yes	        Observer's longitude (decimal degrees format)
observer_alt	float   Yes	        Observer's altitude above sea level in meters
days            integer Yes	        Number of days of prediction (max 10)
min_elevation   integer Yes	        The minimum elevation acceptable for the highest altitude point of the pass (degrees)
```

In [19]:
paste("https://www.n2yo.com/rest/v1/satellite/radiopasses",
                                          satellite_norad_id, station_latitude, station_longitude,
                                          station_elevation, 10, minimum_observable_elevation, n2yo_api_key,
                                          sep="/")

[1] "https://www.n2yo.com/rest/v1/satellite/radiopasses/40069/-29.53/30.25/1167/10/20/&apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ"

In [20]:
satellite_data <- possibly_get_data(paste("https://www.n2yo.com/rest/v1/satellite/radiopasses",
                                          satellite_norad_id, station_latitude, station_longitude,
                                          station_elevation, 10, minimum_observable_elevation, n2yo_api_key,
                                          sep="/"), 8)

In [21]:
satellite_content <- fromJSON(rawToChar(satellite_data$content))

In [22]:
assert_that(!is.null(satellite_content$passes),
        msg = paste("\nUnable to update orbital data from n2yo.com",
                   "\nstatus code:", satellite_data$status_code, 
                   "\nerror messge:", satellite_content$error))

[1] TRUE

In [23]:
assert_that(nrow(satellite_content$passes) > 0,
           msg = "Updated orbital data from n2yo.com is empty")

[1] TRUE

In [24]:
latest_data_date <- max(satellite_data_df$startUTC)

In [25]:
satellite_content %<>% 
    pluck("passes") %>% 
    as_tibble %>%
    filter(startUTC > latest_data_date)

For `mlrpt` need arguments in the form of `mlrpt $startTime-$stopTime -t $duration`. Start and stop time are in the format HH:MM in **UTC**. Duration is how long the command needs to run (in **minutes**) from start to shut-down. Make it 3 minutes longer than the time of the satellite pass to give time for start-up, and at the end, decoding and saving images.

Also need the start time in **local** timezone to schedule `mlrpt` using `systemd`. Move it back 60 seconds to give `systemd` and `mlrpt` time to start up.

In [26]:
satellite_content %<>% 
    mutate(startUTC = as_datetime(startUTC, tz = "UTC")) %>%
    mutate(endUTC = as_datetime(endUTC, tz = "UTC")) %>%
    mutate(startDate = format(startUTC, "%Y-%m-%d")) %>%
    mutate(startTime = format(startUTC, "%H:%M")) %>%
    mutate(endDate = format(endUTC, "%Y-%m-%d")) %>%
    mutate(endTime = format(endUTC, "%H:%M")) %>%
    mutate(localStartTime = format(as_datetime(startUTC, tz = "Africa/Johannesburg") - 60, "%H:%M")) %>% 
    mutate(duration = as.numeric(hm(format(format="%H:%M", endTime)) -
                                hm(format(format="%H:%M",startTime)) + 180) / 60)

Note: method with signature ‘Period#ANY’ chosen for function ‘-’,
 target signature ‘Period#Period’.
 "ANY#Period" would also be valid


In [27]:
satellite_data_df <- bind_rows(satellite_data_df, satellite_content)

In [28]:
saveRDS(satellite_data_df, "satellite_data_df.rds")